In [1]:

import pandas as pd
from datetime import datetime
import geojson
from math import radians, sin, cos, asin, sqrt, floor, ceil 
from scipy.spatial.distance import cdist
import numpy as np
import modules.data_prep as data_prep
import modules.visualisation as viz
from modules.model import EarthquakeAnalyzer
from modules.config import START_MONTH, START_YEAR, END_MONTH, END_YEAR




In [2]:
analyzer = EarthquakeAnalyzer(download_path="./earthquake_data")
files = analyzer.query_period(start_year=START_YEAR, start_month=START_MONTH, end_year=END_YEAR, end_month=END_MONTH)
data = analyzer.extract_data(files)
data = data_prep.extract_cities(data)
features_df, filtered_features, gj = data_prep.load_and_filter_faults(data)
data = data_prep.match_faults_to_earthquakes(data, features_df)
data['timestamp_dt'] = pd.to_datetime(data['timestamp'], errors='coerce')
data = data_prep.calculate_distance_by_m_and_km(features_df, data)

✓ Already exists: 2025-10
✓ Refreshed: 2025-11
Total earthquakes: 9796
Total features: 16195
Filtered features: 1060


In [3]:
filtered = data_prep.filter_by_time(data, start='2025-11-14', end='2025-11-15')


In [4]:
pd.set_option('display.max_columns', None)

In [5]:
map = viz.generate_map(filtered, filtered_features, gj)

In [6]:
ct = str(datetime.now()).replace('-','_').replace(':','_')[:-10]
output_path = f'output_maps/Map_{START_MONTH}.{START_YEAR}_{END_MONTH}.{END_YEAR}_{ct}.html' 

map.save(output_path)